In [1]:
import sys
import os
sys.path.append('/Users/mbassalaev/Desktop/audio_project/Spotter_recognition/')

import pandas as pd
import pickle
import numpy as np

from keras import optimizers
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from service.data_augmentation import make_batch, load_background
from models.crnn import CRNN_model

In [17]:
filename = os.path.abspath('../../../Data_spotter_mfcc_40')
outfile = open(filename,'rb')
X = pickle.load(outfile)
outfile.close()

In [2]:
df = pd.read_csv(os.path.abspath('../full_df'))

In [13]:
df_train, df_test = train_test_split(df, shuffle=True, test_size=6464, train_size = 58240, random_state=42)

In [ ]:
_, X_test, _, Y_test = train_test_split(X
                                                    , df.word
                                                    , test_size=6464
                                                    , train_size = 58240
                                                    , shuffle=True
                                                    , random_state=42)

In [4]:
model = CRNN_model((40, 61, 1), 11)

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(64, 40, 61, 1)]         0         
_________________________________________________________________
conv2d (Conv2D)              (64, 38, 59, 16)          160       
_________________________________________________________________
conv2d_1 (Conv2D)            (64, 34, 57, 16)          3856      
_________________________________________________________________
reshape (Reshape)            (64, 34, 912)             0         
_________________________________________________________________
gru (GRU)                    (64, 256)                 898560    
_________________________________________________________________
flatten (Flatten)            (64, 256)                 0         
_________________________________________________________________
dropout (Dropout)            (64, 256)                 0     

In [6]:
opt = optimizers.Adam(0.001)
model.compile(optimizer = opt, loss=CategoricalCrossentropy(from_logits=True), metrics = ["accuracy"])

In [7]:
filename = os.path.abspath('../../spotter_data_service/lb')
outfile = open(filename,'rb')
lb = pickle.load(outfile)
outfile.close()

In [8]:
background_folder = '/Users/mbassalaev/Desktop/spotter_data/_background_noise_/'

In [11]:
train_metrics = []
test_metrics = []

noise_to_data = load_background(background_folder)

p_aug = {
    'gaussian': 1,
    'stretch': 0.3,
    'pitch_shift': 0.3,
    'shift': 0.5,
    'clipping': 0.3,
}

In [12]:
for step in range(0, 3000):
    X, Y = make_batch(df_train, lb, background_folder, noise_to_data, 64, 22050, p_aug)
    train_step_metric =  model.train_on_batch(X, Y)
    if step > 0 && step % 50 == 0:
        test_step_metric = model.evaluate(X_test, Y_test, batch_size=64)

(64, 40, 61, 1) (64, 11)
[2.018772602081299, 0.34375]
